# Utilisation de données du RP pour requêter SIRENE**Les données du recensement doivent avoir été récupérées et placées** dans le même dossier que le notebook pour son exécution en l'état.

## Import des librairies

Import de la librairie pandas (cf.[tuto panda](./hackathon-tuto-csv.ipynb) et [documentation officielle](https://pandas.pydata.org/pandas-docs/stable/))

In [8]:
import pandas as pd

Import de la librairie requests (cf.[tuto requete API SIRENE](./tuto-api-sirene.ipynb))

In [9]:
import requests

## Lecture du fichier de données RP
En plus de l'usuel chemin vers le fichier de données, deux paramètres sont utilisés :
* sep : permet de spécifier le séparateur utilisé dans le fichier (ici la tabulation)
* error_bad_lines : permet d'échapper les lignes qui ont plus de colonnes que spécifiées dans la 1ère ligne

In [4]:
data = pd.read_csv("./dataRP.txt",sep='\t' , error_bad_lines=False )

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (12,14,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(data)

                                                    RS_X ACTET_C ACTET_C_C  \
0                                        MIGUEL SERVICES   4322A     3700Z   
1                               MAITRE GUILLERON NOTAIRE   6910Z     6910Z   
2                                  CHEQUE EMPLOI SERVICE   9700Z     3700Z   
3                                       IEMTHESE AGEPAPH   8710B     8710B   
4                                ECOLE PRIMAIRE (VENSAT)   8520Z     8411Z   
5                                                   AGIR   8010Z     6920Z   
6                                              LIMAGRAIN   5210B     5210B   
7                                               APAJH 03   8720A     8720A   
8                                     GAEC DE LA PINARDE   0150Z     0150Z   
9                                     GAEC DES DEUX PONT   0150Z     0150Z   
10                                    GAEC DES DEUX PONT   0150Z     0150Z   
11                                       FRANCE BOISSONS   4634Z

Forcément, c'est un grand jeu de données, pas très lisible... mais au moins l'aura-t-on vu une fois (deux ?). 
Contentons-nous pour l'instant de lister les colonnes.

In [10]:
list(data)

['RS_X',
 'ACTET_C',
 'ACTET_C_C',
 'ACTET_X',
 'ILT_X',
 'NOMVOI_X',
 'NUMVOI_X',
 'TYPEVOI_X',
 'BISTER_X',
 'CPLADR_X',
 'CLT_C_C',
 'CLT_X',
 'DLT_X',
 'PLT_X',
 'DEPCOM_CODE',
 'I_MCA_C',
 'I_REPRISE_ACT',
 'I_SIRET_M',
 'NOTE_MCA_ADR_C',
 'NOTE_MCA_NOM_C',
 'PROFA_X',
 'PROFI_X',
 'PROFS_X',
 'VARDOMPART_X',
 'SIRET_DEC',
 'SIRETC',
 'SIRETM']

## Fonctions pour requêter API SIRENE
Pour l'exemple, on définit des fonctions afin de requêter plus facilement l'API.

In [19]:
url = "https://prototype.api-sirene.insee.fr/ws"
insee_proxies = {'http': 'http://proxy-rie.http.insee.fr:8080',
              'https': 'http://proxy-rie.http.insee.fr:8080'}

def build_siret_query (rs):
    siret_query = "/siret/?q=Denomination:{}"
    return url + siret_query.format(rs)

def build_siren_query (rs):
    siren_query = "/siren?q=periode(Denomination:{})"
    return url + siren_query.format(rs)

def search_siret_rs(rs):
    return requests.get(build_siret_query(rs),proxies=insee_proxies)

def search_siren_rs(rs):
    return requests.get(build_siren_query(rs),proxies=insee_proxies)

## Requête à partir des données du RP

Afin qu'il reste des choses à faire le jour du hackathon, on va se contenter de choisir arbitrairement un enregistrement dans les données du RP (le 5ème, outre que c'est mon chiffre fétiche, c'est un cas qui marche bien) et on va habilement s'appuyer sur la valeur de la colonne RS_X (son nom sonne bien)

In [17]:
cinquieme = data.iloc[5]
rs_du_cinquieme = cinquieme['RS_X']
print(rs_du_cinquieme)

AGIR


Il ne reste plus qu'à faire la requête.

In [21]:
requete = search_siret_rs(rs_du_cinquieme)
requete.raise_for_status()

In [ ]:
Maintenant, il ne reste plus qu'à analyser le résultat pour choisir le bon "match"... ça va devenir indigeste le json, mais promis c'est terminé.

In [23]:
response_data=requete.json()
print(response_data)

{'Header': {'Statut': 200, 'Message': 'ok', 'Total': 2255, 'Debut': 0, 'Nombre': 20}, 'Etablissements': [{'Siren': '353816820', 'Nic': '00024', 'StatutDiffusionEtablissement': 'O', 'DateCreationEtablissement': None, 'NombrePeriodes': 1, 'UniteLegale': {'StatutDiffusionUniteLegale': 'O', 'DateCreationUniteLegale': '1990-02-01', 'Sigle': None, 'Sexe': None, 'NomUsage': None, 'Prenom1': None, 'Prenom2': None, 'Prenom3': None, 'Prenom4': None, 'PrenomUsuel': None, 'CaractereEmployeur': 'N', 'UnitePurgee': True, 'EtatAdministratif': 'C', 'Nom': None, 'Denomination': 'AGIR', 'ActivitePrincipale': '70.1F', 'LibelleActivite': 'Marchands de biens immobiliers', 'NomenclatureActivite': 'NAFRev1', 'CategorieJuridique': '5499', 'CategorieProfessionnelle': None, 'NICSiege': '00024'}, 'Adresse': {'ComplementAdresseEtablissement': None, 'NumeroVoieEtablissement': '28', 'IndiceRepetitionEtablissement': None, 'TypeVoieEtablissement': 'AV', 'LibelleVoieEtablissement': 'DU BEZET', 'CodePostalEtablissement

Mais, moi je ne sais pas comment faire ça... une idée ?